# 02443 - Computer Exercise 2: Sampling from Discrete Distributions

In [10]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import plotly.io as pio
pio.templates.default = "plotly_dark"

from utils import chi_sq_test, kolmogorov_smirnov_test

## Part 1 - Simulation using $p$

In [11]:
# Set p to a fixed value on the open interval between 0 and 1
p = 0.01
N = 10000

# Generate uniform random numbers
U = np.random.uniform(0, 1, N)

# Use formula from lecture 3 slide 9 to get geometric distributed discrete numbers
X = np.ceil(np.log(U)/np.log(1-p))

# Histogram
fig = px.histogram(x=X, nbins=50)
fig.update_layout(
    title=f"Geometric Distribution with p={p}",
    xaxis_title="Number of Trials",
    yaxis_title="Frequency",
    width=600,
    height=400,
    bargap=0.1
)
fig.show()

## Part 2 - Simulating 6-point distribution

In [12]:
ps = np.array([7/48, 5/48, 1/8, 1/16, 1/4, 5/16])

### (a) Direct method

In [13]:
def direct(n, ps):
    # Generate uniform random numbers
    U = np.random.rand(N)

    # Convert to discrete random numbers using the given probabilities
    X = np.searchsorted(np.cumsum(ps), U)

    return X

### (b) Rejection method

In [14]:
def rejection(n, ps):
    c = max(ps)
    k = len(ps)

    X = np.zeros(n, dtype=int)
    for i in range(len(X)):
        while True: # Could theoretically run forever...
            U1, U2 = np.random.rand(2)
            I = np.floor(k * U1).astype(int)
            if U2 <= ps[I]/c:
                X[i] = I + 1
                break
    
    return X

### (c) Alias method

In [15]:
def alias(N, ps):
    k = len(ps)

    # Generating Alias tables
    L = np.arange(k)
    F = k*ps
    G = np.where(F >= 1)[0]
    S = np.where(F <= 1)[0]

    while len(S) != 0:
        i = G[0]
        j = S[0]
        L[j] = i
        F[i] -= (1 - F[j])
        if F[i] < 1 - np.finfo(float).eps:
            G = np.delete(G, 0)
            S = np.append(S, i)
        S = np.delete(S, 0)

    # Computing values
    X = np.zeros(N, dtype=int)

    # Generate random numbers
    U1 = np.random.rand(N)
    U2 = np.random.rand(N)

    # Perform Alias method
    I = np.array(np.floor(k * U1)).astype(int)
    mask = U2 <= F[I]
    X[mask] = I[mask] + 1
    X[~mask] = L[I[~mask]] + 1

    return X

### Part 3 - Comparison

In [16]:
X_d = direct(N, ps)
X_r = rejection(N, ps)
X_a = alias(N, ps)


# Create subplot
fig = sp.make_subplots(rows=1, cols=4, subplot_titles=(
    "True probabilities", "Direct Method", "Rejection Method", "Alias Method"
    ))

# True probabilities
fig.add_trace(go.Bar(x=np.arange(1, 7), y=ps, marker_color='green'), row=1, col=1)

# Direct method
fig.add_trace(go.Histogram(x=X_d, histnorm='probability density', marker_color='orange'), row=1, col=2)

# Rejection method
fig.add_trace(go.Histogram(x=X_r, histnorm='probability density', marker_color='blue'), row=1, col=3)

# Alias method
fig.add_trace(go.Histogram(x=X_a, histnorm='probability density', marker_color='red'), row=1, col=4)

# Update layout
fig.update_layout(
    height=400,
    width=1000,
    title_text="Histograms",
    showlegend=False,
    bargap=0.1
)

fig.show()

### Part 4 - Pros and Cons of the different methods